In [1]:
!pip install SentencePiece av

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 35.2 MB/s eta 0:00:00


In [2]:
import av
import numpy as np
import torch
from transformers import AutoTokenizer,AutoImageProcessor,VisionEncoderDecoderModel

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
image_processor = AutoImageProcessor.from_pretrained('MCG-NJU/videomae-base')
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = VisionEncoderDecoderModel.from_pretrained("Neleac/timesformer-gpt2-video-captioning").to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/41.2k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [22]:
video_path = "Nature+Beautiful+short+video+720p+HD.mp4"
container = av.open(video_path)

In [23]:
print(container.streams.video[0].frames)
print(model.config.encoder.num_frames)

544
8


In [24]:
seg_len = container.streams.video[0].frames
clip_len = model.config.encoder.num_frames
indices = set(np.linspace(0, seg_len, num=clip_len, endpoint=False).astype(np.int64))
print(indices)
frames = []
container.seek(0)

{0, 68, 136, 204, 272, 340, 408, 476}


In [25]:
print(container.decode(video=0))

In [26]:
for i, frame in enumerate(container.decode(video=0)):
    if i in indices:
        frames.append(frame.to_ndarray(format="rgb24"))

In [27]:
gen_kwargs = {
    "min_length": 10,
    "max_length": 20,
    "num_beams": 8,
}

In [34]:
pixel_values = image_processor(frames,return_tensors='pt').pixel_values.to(device)
tokens = model.generate(pixel_values,**gen_kwargs)
caption = tokenizer.batch_decode(tokens,skip_special_tokens=True)[0]
print(caption)

A group of people are walking on the edge of a mountain.


In [37]:
from transformers import M2M100ForConditionalGeneration,M2M100Tokenizer

In [39]:
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

In [41]:
tokenizer.src_lang = "en"
encoded_persian = tokenizer(caption,return_tensors='pt')
generated_tokens = model.generate(**encoded_persian,forced_bos_token_id=tokenizer.get_lang_id("fa"))
tokenizer.batch_decode(generated_tokens,skip_special_tokens=True)[0]

'گروهی از مردم در کنار یک کوه راه می روند.'